In [26]:
# Import necessary libraries
from datasets import load_dataset
import pandas as pd

In [27]:
# Load the dataset
climate_fever_dataset = load_dataset("climate_fever")

# Convert the train dataset to pandas DataFrame
climate_fever = pd.DataFrame(climate_fever_dataset['test'])

In [28]:
climate_fever.head()

,claim_id,claim,claim_label,evidences
0,0,Global warming is driving polar bears toward e...,0,[{'evidence_id': 'Extinction risk from global ...
1,5,The sun has gone into ‘lockdown’ which could c...,0,"[{'evidence_id': 'Famine:386', 'evidence_label..."
2,6,The polar bear population has been growing.,1,"[{'evidence_id': 'Polar bear:1332', 'evidence_..."
3,9,Ironic' study finds more CO2 has slightly cool...,1,"[{'evidence_id': 'Atmosphere of Mars:131', 'ev..."
4,10,Human additions of CO2 are in the margin of er...,1,[{'evidence_id': 'Carbon dioxide in Earth's at...


In [29]:
#drop the columns evidence and claim_id
climate_fever = climate_fever.drop(['evidences', 'claim_id'], axis=1)

#rename the column claim_label to label
climate_fever = climate_fever.rename(columns={'claim_label': 'label'})

#replace label values 0 with true and 1, 2,3 with false
climate_fever['label'] = climate_fever['label'].replace(0, 'true')
climate_fever['label'] = climate_fever['label'].replace([1, 2, 3], 'false')

#replace true with 1 and false with 0
climate_fever['label'] = climate_fever['label'].replace('true', 1)
climate_fever['label'] = climate_fever['label'].replace('false', 0)

climate_fever.head()

C:\Users\MONSTER\AppData\Local\Temp\ipykernel_27688\3055851467.py:13: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  climate_fever['label'] = climate_fever['label'].replace('false', 0)


,claim,label
0,Global warming is driving polar bears toward e...,1
1,The sun has gone into ‘lockdown’ which could c...,1
2,The polar bear population has been growing.,0
3,Ironic' study finds more CO2 has slightly cool...,0
4,Human additions of CO2 are in the margin of er...,0


In [30]:
science_dataset = climate_fever

In [31]:
#drop duplicates
science_dataset = science_dataset.drop_duplicates()

#drop rows with missing values
science_dataset = science_dataset.dropna()

In [32]:
#display details of merged_df such as shape, info, describe, head unique values in label column
print(science_dataset.shape)
print(science_dataset.info())
print(science_dataset.describe())
print(science_dataset.head())
print(science_dataset['label'].unique())

#display average claim length
science_dataset['claim_length'] = science_dataset['claim'].apply(lambda x: len(x.split()))
print(science_dataset['claim_length'].mean())

(1535, 2)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1535 entries, 0 to 1534
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   claim   1535 non-null   object
 1   label   1535 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 24.1+ KB
None
             label
count  1535.000000
mean      0.426059
std       0.494664
min       0.000000
25%       0.000000
50%       0.000000
75%       1.000000
max       1.000000
                                               claim  label
0  Global warming is driving polar bears toward e...      1
1  The sun has gone into ‘lockdown’ which could c...      1
2        The polar bear population has been growing.      0
3  Ironic' study finds more CO2 has slightly cool...      0
4  Human additions of CO2 are in the margin of er...      0
[1 0]
20.190879478827362


In [33]:
#Remove punctuations
#Convert text to tokens
#Remove tokens of length less than or equal to 3
#Remove stopwords using NLTK corpus stopwords list to match
#Apply lemmatization

import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

def clean_text(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if len(word) > 3]
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(tokens)

science_dataset['claim'] = science_dataset['claim'].apply(clean_text)
science_dataset.head()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\MONSTER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\MONSTER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\MONSTER\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,claim,label,claim_length
0,global warming driving polar bear toward extin...,1,8
1,gone lockdown could cause freezing weather ear...,1,16
2,polar bear population growing,0,7
3,ironic study find slightly cooled planet,0,10
4,human addition margin error current measuremen...,0,34


In [34]:
#display average claim length
science_dataset['claim_length'] = science_dataset['claim'].apply(lambda x: len(x.split()))
print(science_dataset['claim_length'].mean())
science_dataset = science_dataset.drop(columns=['claim_length'])

11.013029315960912


In [35]:
#save to preprocessed_datasets folder
science_dataset.to_csv('../preprocessed_datasets/science_dataset.csv', index=False)